In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy.stats import norm
import matplotlib.mlab as mlab
import seaborn as sns
import bokeh
import keras
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
df = pd.read_csv('../input/properties_2016.csv')
df1 = pd.read_csv('../input/train_2016_v2.csv')
sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
dfmerge =df.merge(df1, on='parcelid', how ='left')
dfmerge =dfmerge[np.isfinite(dfmerge['logerror'])]
dfmerge.head(n=10)

#sample['parcelid'] = sample['ParcelId']
#print("Merge Sample with property data :")
#df_test = sample.merge(prop, on='parcelid', how='left')

In [ ]:

trainx = dfmerge.drop(['logerror','parcelid','architecturalstyletypeid','basementsqft'
                      ,'buildingclasstypeid','calculatedbathnbr','decktypeid','finishedfloor1squarefeet'
                      ,'finishedsquarefeet13'
                      ,'finishedsquarefeet15','finishedsquarefeet50','finishedsquarefeet6'
                      ,'fips','fullbathcnt','garagecarcnt','hashottuborspa'
                      ,'latitude','longitude','lotsizesquarefeet','poolsizesum'
                      ,'pooltypeid10','pooltypeid2','pooltypeid7','propertycountylandusecode'
                      ,'propertylandusetypeid','propertyzoningdesc','rawcensustractandblock'
                      ,'regionidzip','storytypeid','threequarterbathnbr','typeconstructiontypeid',
                      'yardbuildingsqft17','yardbuildingsqft26','fireplaceflag','assessmentyear'
                      ,'landtaxvaluedollarcnt','taxdelinquencyflag','taxdelinquencyyear'
                      ,'censustractandblock','transactiondate'],axis=1,inplace=False)

trainy= dfmerge['logerror']
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
#optional is default mean#
imputer= Imputer()
imputer.fit(trainx.iloc[:, :])
x_train = imputer.transform(trainx.iloc[:, :])
sc = StandardScaler()
x_train = sc.fit_transform(x_train)

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm
from keras.layers import Dropout

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(x_train, trainy, test_size=0.2)
print (Xtrain.shape, ytrain.shape)
print(Xtest.shape, ytest.shape)


In [ ]:
neurons = 1
model = Sequential()
model.add(Dense(neurons, input_dim=20, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(4)))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.add(Dense(1, kernel_initializer='normal', activation='linear'))
model.add(Dense(1, kernel_initializer='normal', activation='linear'))
model.compile(loss='mean_absolute_error', optimizer='adam')

In [ ]:
model.fit(x_train, trainy, epochs=10, batch_size=10)

In [ ]:
predictions = model.predict(Xtest)
print(predictions)

In [ ]:
model.fit(Xtest, ytest, epochs=10, batch_size=10)

In [ ]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
seed =24

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(neurons, input_dim=20, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    #model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    #model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
estimator = KerasRegressor(build_fn=create_model, nb_epoch=10, batch_size=100, verbose=False)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, Xtrain, ytrain, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
from sklearn.metrics import accuracy_score
estimator.fit(x_train, trainy)
#prediction = estimator.predict(Xtest)
estimator.score(Xtest, ytest)
prediction = estimator.predict(Xtest)


In [ ]:
parameters = {'n_estimators':[10,15,20],'n_jobs':[-1],'oob_score':[False]}
model = RandomForestRegressor()
grid = GridSearchCV(model,param_grid=parameters,scoring='neg_mean_absolute_error',cv=3)
grid.fit(Xtest,ytest)
#model.fit(Xtrain,ytrain)

cv_results = pd.DataFrame(grid.cv_results_)
print(cv_results[["param_n_estimators","mean_test_score","std_test_score"]])

In [ ]:
#submit this with the ending #
y_pred = prediction.flatten()

#output = pd.DataFrame({'ParcelId': properties['parcelid'].astype(np.int32),
output = pd.DataFrame({'ParcelId': df['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})

cols = output.columns.tolist()
cols = cols[-1:] + cols[:-1]
output = output[cols]

print( "\nWriting results to disk:" )
output.to_csv('testing'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)

print( "\nFinished!" )

In [ ]:
estimators = []
seed =24
#model.fit(x_train, trainy, epochs=2, batch_size=10)

estimators.append(('mlp', KerasRegressor(build_fn=create_model, epochs=10, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline,x_train, trainy, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))